In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import image as mpimg
import shutil
from google.colab import drive

drive.mount('/content/drive')

In [61]:
scores_csv = pd.read_csv('/content/drive/MyDrive/galaxy-segmentation-project/scores.csv')

iterables = [['Accuracy', 'Precision', 'Sensitivity', 'Specificity', 'Jaccard'], ['th=1', 'th=2', 'th=3', 'th=4'], ['Only spirals', 'All images']]
headers = pd.MultiIndex.from_product(iterables)

df_scores = pd.DataFrame(columns=headers, index=range(len(scores_csv)))
df_scores.insert(0, 'Galaxy ID', 0)

df_scores[:] = scores_csv[0:]

In [62]:
temp = df_scores['Jaccard', 'th=3', 'All images'].dropna().sort_values(ascending=False)
sorted_gal = df_scores['Galaxy ID'].loc[temp.index]
best_ids = sorted_gal[:30].values
worst_ids = sorted_gal[-30:].values

# Best predictions

In [ ]:
for gal in best_ids:
  path = f'/content/drive/MyDrive/galaxy-segmentation-project/Binary masks figures/'
  shutil.copy(f'{path}{gal}.jpg', f'{path}Best predictions')
  image = mpimg.imread(f'/content/drive/MyDrive/galaxy-segmentation-project/Binary masks figures/{gal}.jpg')
  plt.figure(figsize=(22,13))
  plt.axis('off')
  plt.imshow(image)

# Worst predictions

In [ ]:
for gal in worst_ids:
  path = f'/content/drive/MyDrive/galaxy-segmentation-project/Binary masks figures/'
  shutil.copy(f'{path}{gal}.jpg', f'{path}Worst predictions')
  image = mpimg.imread(f'/content/drive/MyDrive/galaxy-segmentation-project/Binary masks figures/{gal}.jpg')
  plt.figure(figsize=(22,13))
  plt.axis('off')
  plt.imshow(image)